In [ ]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [1]:
import gradio as gr

In [3]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [4]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. 
# If you're at work on on a work network, I suggest skip this test.

# gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("kshitij", "test1234"))


In [5]:
message_input = gr.Textbox(label="Your message", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title='Shout',
    inputs=[message_input],
    outputs = [message_output],
    examples=["hello", "namaste"],
    flagging_mode="never"
)
view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input i am yelling !!
Shout has been called with input namaste


In [ ]:
from openai import OpenAI
openai = OpenAI()

system_message = " Your are a extremely rude assistant that responds in Markdown without code blocks"

def message_gpt(prompt):
    messages = [
        {'role': 'system', 'content' : system_message},
        {'role': 'user', 'content' : prompt}
        ]
    response = openai.chat.completions.create(model='gpt-4.1-mini', messages=messages)
    return response.choices[0].message.content

In [10]:
message_input = gr.Textbox(label="Your message", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=message_gpt,
    title='Shout',
    inputs=[message_input],
    outputs = [message_output],
    examples=["hello", "namaste"],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [16]:
system_message = " Your are a extremely rude assistant that responds in Markdown without code blocks"

message_input = gr.Textbox(label="Your message", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gpt,
    title='GPT',
    inputs=[message_input],
    outputs = [message_output],
    examples=[
        "Explain the Chi-square test to a layperson",
        "Explain the ANOVA test to a layperson"
    ],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [26]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [27]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [29]:
gpt_model = "gpt-4.1-mini"
gemini_model = "gemini-2.5-flash-lite"

openai = OpenAI()
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [ ]:
system_message = " Your are a extremely cordial and helpful assistant that responds in Markdown without code blocks"

def stream_gpt(prompt):
    messages = [
        {'role': 'system', 'content' : system_message},
        {'role': 'user', 'content' : prompt}
    ]
    stream = openai.chat.completions.create(model='gpt-4.1-mini', messages=messages,stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [30]:
# message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT", lines=7)
# message_output = gr.Markdown(label="Response:")

# view = gr.Interface(
#     fn=stream_gpt,
#     title = "GPT chatbot",
#     inputs = [message_input],
#     outputs = [message_output],
#     flagging_mode="never"
# )
# view.launch(inbrowser=True)

In [43]:
system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""

In [44]:
def stream_gpt(prompt):
    messages = [
        {'role': 'system', 'content' : system_message},
        {'role': 'user', 'content' : prompt}
    ]
    stream = openai.chat.completions.create(model=gpt_model, messages=messages,stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [45]:
def stream_gemini(prompt):
    messages = [
        {'role': 'system', 'content' : system_message},
        {'role': 'user', 'content' : prompt}
    ]
    stream = gemini.chat.completions.create(model=gemini_model, messages=messages,stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [46]:
from scraper import fetch_website_contents

def stream_brochure(company_name, url , model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += fetch_website_contents(url)
    if model=='GPT':
        result = stream_gpt(prompt)
    elif model == 'Gemini':
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result
    

In [ ]:
name_input = gr.Textbox(label= "Company Name:")
url_input = gr.Textbox(label="Landing page URL including http:// or https://")
model_selector = gr.Dropdown(["GPT", "Gemini"], label='Select model', value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_brochure,
    inputs = [name_input, url_input, model_selector],
    outputs = [message_output],
    examples=[
            ["Hugging Face", "https://huggingface.co", "GPT"],
            ["Edward Donner", "https://edwarddonner.com", "Claude"]
        ], 
    flagging_mode="never")
view.launch(inbrowser=True)

/Users/kshitijchaudhary/llm_eng/llm_eng/.venv/lib/python3.13/site-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Claude or set allow_custom_value=True.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
